In [1]:
# This script is used to create Figure 1 in the paper
# It requires running TIMEVIEW_interface_only.sh or Table_3.sh first to generate the results

%matplotlib inline
import sys
sys.path.append('../../')
from timeview.visualize import expert_tts_plot, grid_tts_plot
from experiments.datasets import load_dataset
from experiments.benchmark import load_column_transformer, create_benchmark_datasets_if_not_exist
from timeview.lit_module import load_model
from experiments.baselines import YNormalizer
from experiments.analysis.analysis_utils import find_results

# dataset_name = "synthetic_tumor_wilkerson_1"
# dataset_name = "airfoil_log"
# dataset_name = "stress-strain-curves"
dataset_name = "flchain_1000"
model_name = "TTS"
root = "/dataNAS/people/sostm/TIMEVIEW/experiments"

create_benchmark_datasets_if_not_exist(dataset_description_path=f"{root}/dataset_descriptions")

results = find_results(dataset_name, model_name, results_dir=f'{root}/benchmarks', summary_filename=f'{root}/benchmarks/summary.json',)

if len(results) == 0:
    print(f"No results found for {dataset_name} and {model_name}")
    print("Make sure you run your experiments from ../run_scripts")
elif len(results) > 1:
    print("Multiple results found for the given dataset and model")
    print("We take the last one but it may produce unexpected results")

timestamp = results[-1]

litmodel = load_model(timestamp, seed=661058651, benchmarks_folder=f"{root}/benchmarks")
dataset = load_dataset(dataset_name, dataset_description_path=f"{root}/dataset_descriptions", data_folder=f"{root}/data")
column_transformer = load_column_transformer(timestamp, benchmarks_dir=f"{root}/benchmarks")
y_normalizer = YNormalizer.load_from_benchmark(timestamp, model_name, benchmark_dir=f"{root}/benchmarks")

Multiple results found for the given dataset and model
We take the last one but it may produce unexpected results
Overriding data folder with /dataNAS/people/sostm/TIMEVIEW/experiments/data


In [2]:

# expert_tts_plot(litmodel, dataset, (0.2,0.5), n_points=100, figsize=(3.5,3.5), column_transformer=column_transformer, y_normalizer=y_normalizer, display_feature_names=None,default_values=None)

In [3]:
import numpy as np
import pandas as pd
from timeview.basis import BSplineBasis

def tts_inference(litmodel, dataset, feature_combinations, n_points=100, y_normalizer=None, column_transformer=None, return_transition_points=False):
    """
    Perform inference on trajectory time series model for multiple feature combinations.
    
    Parameters:
    -----------
    litmodel : TTSModel
        The trained trajectory time series model
    dataset : Dataset
        The dataset object containing feature information
    feature_combinations : list[dict] or pd.DataFrame
        List of dictionaries or DataFrame containing feature combinations to evaluate
        Each dictionary/row should contain feature names as keys/columns
    n_points : int, optional (default=100)
        Number of points to evaluate the trajectory
    y_normalizer : sklearn transformer, optional
        Transformer to inverse transform the predictions if used during training
    column_transformer : sklearn ColumnTransformer, optional
        Transformer used to preprocess features during training
    return_transition_points : bool, optional (default=False)
        Whether to return transition points in addition to trajectories
        
    Returns:
    --------
    dict with keys:
        't' : ndarray 
            Time points for the trajectories
        'trajectories' : ndarray
            Predicted trajectories for each feature combination
        'transition_points' : list[tuple], optional
            List of (time_points, values) tuples for transition points if return_transition_points=True
    """
    # Convert input to DataFrame if it's a list of dictionaries
    if isinstance(feature_combinations, list):
        feature_combinations = pd.DataFrame(feature_combinations)
    
    # Verify features
    feature_names = dataset.get_feature_names()
    for feature in feature_names:
        if feature not in feature_combinations.columns:
            raise ValueError(f"Missing feature: {feature}")
    
    # Get time points
    time_horizon = litmodel.config.T
    t = np.linspace(0, time_horizon, n_points)
    
    # Initialize BSpline basis
    config = litmodel.config
    bspline = BSplineBasis(
        config.n_basis,
        (0, config.T),
        internal_knots=config.internal_knots
    )
    
    # Transform features if needed
    if column_transformer is not None:
        transformed_features = column_transformer.transform(feature_combinations[feature_names])
    else:
        transformed_features = feature_combinations[feature_names].values
        
    # Get trajectories
    trajectories = []
    transition_points_list = []
    
    for i in range(len(transformed_features)):
        # Get trajectory
        trajectory = litmodel.model.forecast_trajectory(transformed_features[i], t)
        
        # Apply inverse transform if needed
        if y_normalizer is not None:
            trajectory = y_normalizer.inverse_transform(trajectory.reshape(-1, 1)).flatten()
            
        trajectories.append(trajectory)
        
        # Get transition points if requested
        if return_transition_points:
            coeffs = litmodel.model.predict_latent_variables(
                transformed_features[i].reshape(1, -1)
            )
            _, transition_points = bspline.get_template_from_coeffs(coeffs[0, :])
            
            # Get values at transition points
            transition_values = litmodel.model.forecast_trajectory(
                transformed_features[i], 
                np.array(transition_points)
            )
            
            # Apply inverse transform if needed
            if y_normalizer is not None:
                transition_values = y_normalizer.inverse_transform(
                    transition_values.reshape(-1, 1)
                ).flatten()
                
            transition_points_list.append((transition_points, transition_values))
    
    result = {
        't': t,
        'trajectories': np.array(trajectories)
    }
    
    if return_transition_points:
        result['transition_points'] = transition_points_list
        
    return result

In [4]:
settings = [
    {
        "age": 65,
        "creatinine": 1.2,
        "flc.grp": "normal",
        "kappa": 1.5,
        "lambda": 1.0,
        "mgus": 0,
        "sex": "M"
    },
    {
        "age": 75,
        "creatinine": 1.5,
        "flc.grp": "high",
        "kappa": 2.0,
        "lambda": 1.3,
        "mgus": 1,
        "sex": "F"
    }
]

# Get trajectories
results = tts_inference(
    litmodel=litmodel,
    dataset=dataset,
    feature_combinations=settings
)


NameError: name 'your_model' is not defined

In [ ]:
results